# 0. Install Dependencies

In [1]:
# !pip install tensorflow
# !pip install gym
# !pip install keras
# !pip install keras-rl2

# 1. Test Environment with OpenAI Gym

https://www.youtube.com/watch?v=bD6V3rcr_54&list=PLgNJO2hghbmjlE6cuKMws2ejC54BTAaWV&index=3

In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [4]:
import random
import numpy as np
from gym import Env
from gym.spaces import Discrete, Box #Define action and current state environment    

class TaxiChargingEnv(Env):
    def __init__(self):
        # Actions we take: 0 kW, 7 kW, 14 kW, 22 kW
        self.action_space = Discrete(4)
        # State represents battery charge level (kWh)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Initialize state
        self.state = 0
        # Battery capacity
        self.battery_capacity = 100  # kWh
        # Initial battery level
        self.battery_level = random.randint(20, 30)
        # Charging period (2 hours, every 15 minutes = 8 steps)
        self.steps_left = 8
        # Energy demand parameters
        self.mu = 30  # kWh
        self.sigma = 5  # kWh
        # Time coefficient for cost function
        self.alpha = 0.1
    
    def step(self, action):
        charging_rates = [0, 7, 14, 22]  # kW for each action
        power = charging_rates[action]
        
        # Calculate cost: charging cost (t,p) = ∑ 𝛼𝑡 * e^𝑝
        cost = self.alpha * np.exp(power)
        
        # Update battery level
        self.battery_level = min(self.battery_level + power / 4, self.battery_capacity) #each time 15 min, 60/15=4
        
        # Reduce the steps left by 1
        self.steps_left -= 1
        
        # If steps are finished, calculate if the battery meets the demand
        if self.steps_left == 0:
            demand = np.random.normal(self.mu, self.sigma)
            if self.battery_level >= demand:
                reward = -cost  # Minimize cost, no penalty for meeting demand
            else:
                reward = -cost - 1000  # High penalty for not meeting demand
            done = True
        else:
            reward = -cost  # Incur cost for charging, continue episode
            done = False
        
        # State is the current battery level
        self.state = self.battery_level
        
        # Info dictionary
        info = {
            "battery_level": self.battery_level,
            "cost": cost
        }
        
        return self.state, reward, done, info
    
    def reset(self):
        # Reset battery level
        self.battery_level = random.randint(20, 30)
        # Reset steps left
        self.steps_left = 8
        # Reset state
        self.state = self.battery_level
        return self.state
    
    def render(self):
        # For visualization, if needed
        pass


In [5]:
# Create the environment
env = TaxiChargingEnv()

/Users/hsienpanghsieh/miniconda3/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [7]:
env.action_space.sample()

1

In [10]:
env.observation_space.sample()

array([22.892267], dtype=float32)

In [11]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-361000.9118811188
Episode:2 Score:-1075714703.686258
Episode:3 Score:-601521.5687140741
Episode:4 Score:-358852285.42504036
Episode:5 Score:-1433965248.4159527
Episode:6 Score:-358612764.7682073
Episode:7 Score:-1075834744.8880427
Episode:8 Score:-358732134.6599397
Episode:9 Score:-717223309.609783
Episode:10 Score:-717223309.6097832


# 2. Create a Deep Learning Model with Keras

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [8]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [20]:
model = build_model(states, actions)

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [5]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [6]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [25]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: 1.0000
51 episodes - episode_reward: 193.118 [58.000, 200.000] - loss: 5.376 - mae: 39.195 - mean_q: 78.705

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: 1.0000
54 episodes - episode_reward: 185.056 [64.000, 200.000] - loss: 7.553 - mae: 40.527 - mean_q: 81.044

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 51s 5ms/step - reward: 1.0000
52 episodes - episode_reward: 193.462 [33.000, 200.000] - loss: 8.935 - mae: 40.588 - mean_q: 81.322

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: 1.0000
50 episodes - episode_reward: 200.000 [200.000, 200.000] - loss: 11.357 - mae: 41.679 - mean_q: 83.338

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 1.0000
done,

In [27]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [29]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


# 4. Reloading Agent from Memory

In [30]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [31]:
del model
del dqn
del env

In [9]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [10]:
dqn.load_weights('dqn_weights.h5f')

In [11]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
